In [1]:
import json
import os
import shutil
import re
import sqlite3
from string import punctuation
from nltk import sent_tokenize, word_tokenize
from langdetect import detect

##### Как выглядит база данных

In [95]:
corpname = 'ALLPOP'

In [11]:
conn = sqlite3.connect(f'{corpname}.db')
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS author
(id INTEGER PRIMARY KEY AUTOINCREMENT, band text, genre text)
""")
# таблица автор: ключ, название группы, жанр
# таблица песня: номер автора, название, год, альбом, текст, лемматизированный текст
cur.execute("""
CREATE TABLE IF NOT EXISTS song 
(id INTEGER PRIMARY KEY AUTOINCREMENT, author_id int, songname text, year int, album text, lyrics text, lemmatized_lyr text) 
    """)
# таблика токен: номер песни, номер строки, токен(=слово как встретилось в lyrics с сохранением регистра), лемма, часть речи, разбор, позиция в предложении
cur.execute("""
CREATE TABLE IF NOT EXISTS token
(id INTEGER PRIMARY KEY AUTOINCREMENT, song_id int, line int, token text, bast int, lem text, pos text, gram text, in_sent int) 
""")

conn.commit()
conn.close()

##### Список необходимых паттернов

In [2]:
pattern_br = re.compile('\(.+\)')
pattern_encl = re.compile('(.+?)[\?\.\;\:\!\n]+ ?')
pattern_text = re.compile('(.+) \(.+\)')
pattern_date = re.compile('\d{4}')
stripadd = re.compile('.+\]\n')
punct = punctuation + '...' + '``' + '\'\'' + '«' + '»' + '—' + ' '
thrash = ['rmx', 'edit', 'раунд', 'round', 'cover', 'кавер', 'фристайл', 'freestyle', 'live', 'лайв', 'version', 'версия', 'mix', 'remix', 'микс', 'ремикс', 'скит', r'(Скит)','skit', 'демо', 'ark', 'список коллабораций']

##### Разбор

Происходит в mystem консольно

##### Функции

forstem - папка, в которой находятся тексты, которые нужно разобрать + название текстового файла <br>
mystem_path - расположение mystem.exe <br>
output_filename - место, куда сохранится размеченный майстемом файл + назание файла (по умолчанию одинаковое и то, и то, различается форматом

In [3]:
def writeforstem(author_id, idx, song_id, sentences):
    lines_to_parse = '\n'.join([line for line in sentences])
    forstem = os.path.join(r'\Users\mjo\Documents\GitHub\RU-PopCultural-Corpus\forstemming', f'{author_id}\\{author_id}_{idx}_{song_id}.txt')
    with open(forstem, 'w+', encoding='utf-8') as f:
        f.write(lines_to_parse)
    mystem_path = os.path.join(r'\Users\mjo\Desktop', 'mystem')
    output_filename = os.path.join(r'\Users\mjo\Documents\GitHub\RU-PopCultural-Corpus\forstemming',  f'{author_id}\\{author_id}_{idx}_{song_id}.json')
    os.system(f"{mystem_path} {forstem} {output_filename} -n -i -d -s -c --eng-gr --format json")
    return output_filename

In [4]:
def check_title(title):
    check = 0
    title_items = title.split(' ')
    title_items = [item.strip(punctuation).lower() for item in title_items]
    for item in title_items:
        if item in thrash:
            check = 1
    return check

In [5]:
def extract_date(rel_date, rel_ford):
    if rel_date:
        d_text = pattern_date.search(rel_date)
        if d_text:
            date = d_text.group(0)
    elif rel_ford:
        df_text = pattern_date.search(rel_ford)
        if df_text:
            date = df_text.group(0)
        else:
            date = '0'
    else:
        date = '0'
    return int(date)

In [6]:
def get_album(song):
    try:
        if song['album']['name']:
            album = release_from_brakes('name', song['album'])
            return album
    except:
        return 'undefined'

In [7]:
def release_from_brakes(el, obj):
    p_text = pattern_text.search(obj[el])
    if p_text:
        element = p_text.group(1)
    else:
        element = obj[el]
    return element

In [8]:
def get_lyrics(genius_raw):
    if genius_raw:
        without_unicode = genius_raw.replace('\u2005', ' ').replace('\xa0', ' ')  # заменяю не-классические табуляции на классические
        without_comments = stripadd.sub('', without_unicode).replace('\n\n', '\n').strip('\n')  # удаляю текст из скобок [типа Припев 1] | такой текст будет записан в графу lyrics
        lyric_lines = without_comments.splitlines()  #  делю текст на строки
        extracted = find_additional(lyric_lines)  # ищу в строках все предложения
        sentences = get_sentences(extracted)  # получаю список всех предложений текста
        uniq_lists = only_uniqs(sentences)  # теперь в списке каждое предложение может встретиться только 1 раз, убраны любые повторения типа припевов
        return without_comments, uniq_lists

In [9]:
def find_additional(lines):
    extracted_lines = []
    for line in lines:
        if line:
            check_brake = pattern_br.search(line)
            if check_brake:
                additional_line = check_brake.group(0)
                clean_original = line.replace(additional_line, '')
                if clean_original:
                    extracted_lines.append(clean_original)
                extracted_lines.append(additional_line)
            else:
                extracted_lines.append(line)            
    return extracted_lines

In [10]:
def get_sentences(lines):
    sentences = []
    for line in lines:
        sent_list = pattern_encl.findall(line)
        if sent_list:
            for sent in sent_list:
                sentences.append(sent.strip(punct))
        else:
            sentences.append(line.strip(punct))
    return sentences

In [12]:
def clean_words(sentences):
    word_list = []
    for sentence in sentences:
        cleaned_words = [word.strip(punct) for word in word_tokenize(sentence) if word.strip(punct)]
        separated = []
        for i in range(len(cleaned_words)):
            if len(cleaned_words[i].split('-'))>1:
                cl_w = cleaned_words[i].split('-')
                for cl in cl_w:
                    separated.append(cl)
            else:
                separated.append(cleaned_words[i])
        word_list.append(separated)
    return word_list

In [13]:
def only_uniqs(sent_wordlists):
    uniq_list = []
    uniq_lower = []
    for sentence in sent_wordlists:
        if sentence.lower().strip(' ') not in uniq_lower:
            uniq_lower.append(sentence.lower().strip(' '))
            uniq_list.append(sentence)
    return uniq_list

In [14]:
def lem_to_string(lemm_list):
    lemmatized = []
    for sent in lemm_list:
        line_lyric = ' '.join([snt for snt in sent])
        lemmatized.append(line_lyric)
    lemma_lyrics = '\n'.join([line for line in lemmatized])
    return lemma_lyrics

Пляски с mystem и перезаписями

In [15]:
def lyr_to_lem(token_id, filename, song_id):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    num_line = 0
    num_insent = 0
    listsfromsong = []
    sentence_list = []
    for it in text.splitlines():
        line = json.loads(it)
        if line.get('text', [])!=' ': 
            if line.get('text', []) =='\r\n':
                num_insent = 0
                if sentence_list:
                    listsfromsong.append(sentence_list)
                    num_line+=1
                sentence_list = []
            if line.get('analysis', []):
                lemma = line.get('analysis', [])[0]['lex']
                gr = line.get('analysis', [])[0]['gr'].replace('=', ',')
                pos, _ = gr.split(',', 1)
                token = line.get('text')
                if line.get('analysis', [])[0].get('qual',[]):
                    bastard = 1
                else:
                    bastard = 0
                sentence_list.append(lemma) 
                cur.execute('INSERT INTO token VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
                            (token_id, song_id, num_line, token, bastard, lemma, pos, gr, num_insent ))
                conn.commit()
                num_insent +=1
                token_id +=1
    if sentence_list:
        listsfromsong.append(sentence_list)
    if listsfromsong:
        lemmatized = lem_to_string(listsfromsong)
        return token_id, lemmatized
    else:
        return token_id, listsfromsong

### Пример записи корпуса

In [ ]:
author_ids = []
token_id = 0
song_ids = 0

In [ ]:
conn = sqlite3.connect(f'{corpname}.db')
cur = conn.cursor()
for file in ffiles:
    with open(file, 'r', encoding='utf-8') as f:
        artist_json = json.load(f)
    checkfifty = 0  # проверка, что запишется не больше 50 песен на исполнителя; можно удалять, если ограничений нет
    artist = release_from_brakes('name',artist_json).replace('\xa0', ' ')   # может понадобиться заменить '\u200b', если имя исполнителя с маленькой буквы('монеточка')
    author_id = len(author_ids)
    os.makedirs(str(author_id), exist_ok=True)  # папка для исполнителя
    try:
        shutil.move(str(author_id), r'forstemming')  # папка переносится в подпапку "для разметки"
    except:
        pass
    titles = []
    for idx, song in enumerate(artist_json['songs']):
        if checkfifty<50:
            date = extract_date(song['release_date'], song['release_date_for_display'])
            if 2020>date>1989 or date==0:  # если есть какие-то условия по записи в определенные годы
                title = release_from_brakes('title', song)
                if title not in titles:
                    check = check_title(title)
                    if check == 0:
                        song_id = song_ids
                        album = get_album(song)
                        try:
                            lyrics, sentences = get_lyrics(song.get('lyrics', []))  # возвращает (1) очищенные строки с пунктуацией внутри (0_2) список предложений, который нужен дальше
                            output_f = writeforstem(author_id, idx, song_id, sentences)
                            token_id, lemmatized = lyr_to_lem(token_id, output_f, song_id)
                            if lemmatized:  # если получилось сделать разметку, данные о песни записываются в базу
                                cur.execute(  # пишу данные в базу текста
                                    'INSERT INTO song VALUES (?, ?, ?, ?, ?, ?, ?)',
                                    (song_id, author_id,
                                     title, date, album,
                                     lyrics, lemmatized))
                                conn.commit()
                                song_ids +=1
                                checkfifty+=1
                                titles.append(title)
                        except:
                            pass 
    author_ids.append(artist)
    cur.execute('INSERT INTO author VALUES (?, ?, ?)', (author_id, artist, 'альтернатива'))  # здесь еще указывается жанр исполнителя
    conn.commit()
conn.close()

## Сборка корпуса

In [41]:
files = os.listdir('./corp/middlepop')  # папка, в которой находятся ваши файлы .json

In [45]:
ffiles =[os.path.join('./corp/middlepop/', f'{file}') for file in files]

Иногда нужно ограничить количество песен по какому-то условию. Это можно сделать например вот так, сделав два цикла. <br>
Например - 15 песен строго из до 2011 года. Остальные - самые популярные

In [73]:
conn = sqlite3.connect(f'{corpname}.db')
cur = conn.cursor()
for file in f2files:
    with open(file, 'r', encoding='utf-8') as f:
        artist_json = json.load(f)
    checktwenty = 0  # проверка, что запишется не больше 20 песен на исполнителя
    artist = release_from_brakes('name',artist_json).replace('\xa0', ' ')  # может понадобиться заменить '\u200b', если имя исполнителя с маленькой буквы('монеточка')
    author_id = len(author_ids)
    os.makedirs(str(author_id), exist_ok=True)
    try:
        shutil.move(str(author_id), r'forstemming')
    except:
        pass
    titles = []
    for idx, song in enumerate(artist_json['songs']):
        if checktwenty<20:
            date = extract_date(song['release_date'], song['release_date_for_display'])
            if 2010>date>1989 or date==0:
                title = release_from_brakes('title', song)
                if title not in titles:
                    check = check_title(title)
                    if check == 0:
                        song_id = song_ids
                        album = get_album(song)
                        try:
                            lyrics, sentences = get_lyrics(song.get('lyrics', []))  # возвращает (1) очищенные строки с пунктуацией внутри (0_2) список предложений, который нужен дальше
                            if detect(lyrics) == 'ru':
                                output_f = writeforstem(author_id, idx, song_id, sentences)
                                token_id, lemmatized = lyr_to_lem(token_id, output_f, song_id)
                                if lemmatized:
                                    song_ids +=1
                                    cur.execute(  # пишу данные в базу текста
                                        'INSERT INTO song VALUES (?, ?, ?, ?, ?, ?, ?)',
                                        (song_id, author_id,
                                         title, date, album,
                                         lyrics, lemmatized))
                                    conn.commit()
                                    checktwenty+=1   
                                    titles.append(title)
                        except:
                            pass 
    author_ids.append(artist)
    cur.execute('INSERT INTO author VALUES (?, ?, ?)', (author_id, artist, 'поп'))
    conn.commit()
conn.close()

## Вариант с листом excel

In [ ]:
import pandas as pd

In [ ]:
file = 'write2b.xlsx'

In [ ]:
df = pd.read_excel(file)

In [ ]:
songlist = df.values.tolist()

In [94]:
conn = sqlite3.connect(f'{corpname}.db')
cur = conn.cursor()
for song in llist:
    artist = song[1]
    author_id = song[0]
    os.makedirs(str(author_id), exist_ok=True)
    try:
        shutil.move(str(author_id), r'forstemming')
    except:
        pass
    date = song[4]
    title = song[3]
    song_id = song_ids
    album = song[5]
    lyrics, sentences = get_lyrics(song[6])  # возвращает (1) очищенные строки с пунктуацией внутри (0_2) список предложений, который нужен дальше
    output_f = writeforstem(author_id, idx, song_id, sentences)
    token_id, lemmatized = lyr_to_lem(token_id, output_f, song_id)
    if lemmatized:
        song_ids +=1
        cur.execute(
            'INSERT INTO song VALUES (?, ?, ?, ?, ?, ?, ?)',
            (song_id, author_id,
             title, date, album,
             lyrics, lemmatized))
        conn.commit()
conn.close()